# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [51]:
#Your code here
import pandas as pd

df = pd.read_csv('homepage_actions.csv')
print(df.columns)

print(df['group'].unique())
print(df['action'].unique())
print('Number of unique IDs:', len(df['id'].unique()))

df_click = df[df['action'] == 'click']
df_view = df[df['action'] == 'view']

print('Number of views:', len(df_view))
print('Number of clicks:', len(df_click))
print('Viewers that clicked:', sum(df_view['id'].isin(df_click['id'])))


df_control =  df[df['group'] == 'control']
df_experiment =  df[df['group'] == 'experiment']
set1 = set(df_control['id'])
set2 = set(df_experiment['id'])
overlapping_ids = set1.intersection(set2)

print('Overlap between control and experiment group:', len(overlapping_ids))

Index(['timestamp', 'id', 'group', 'action'], dtype='object')
['experiment' 'control']
['view' 'click']
Number of unique IDs: 6328
Number of views: 6328
Number of clicks: 1860
Viewers that clicked: 1860
Overlap between control and experiment group: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [71]:
import numpy as np
from scipy.stats import levene


control = df[df.group=='control'].pivot(index='id', columns='action', values='count')
control = control.fillna(value=0)

experiment = df[df.group=='experiment'].pivot(index='id', columns='action', values='count')
experiment = experiment.fillna(value=0)


print("Sample sizes:\tControl: {}\tExperiment: {}".format(len(control), len(experiment)))
print("Clicks:\tControl: {}\tExperiment: {}".format(control.click.sum(), experiment.click.sum()))
print("Mean clicks:\tControl: {}\tExperiment: {}".format(control.click.mean(), experiment.click.mean()))
print("Variance clicks:\tControl: {}\tExperiment: {}".format(np.var(control.click), \
                                                            np.var(experiment.click)))

# Perform Levene's test
statistic, p_value = levene(control.click, experiment.click)

# Print results
print("Levene's test statistic:", statistic)
print("p-value:", p_value)
alpha = 0.05

if p_value <= alpha:
    print("Variances are significantly different")
else:
    print("Variances are same")
    
# We'll use independent T2 test
# H0: Experimental homepage has the same effect as the control page  
# H1: Experiemntal page is more engaging than the control page 

# Reject H0 if t>0 and p/2 < alpha

from scipy.stats import ttest_ind

# Perform t-test with unequal variances
statistic, p_value = ttest_ind(experiment.click, control.click, equal_var=False)

# Print results
print("T-test statistic:", statistic)
print("p-value:", p_value/2)


Sample sizes:	Control: 3332	Experiment: 2996
Clicks:	Control: 932.0	Experiment: 928.0
Mean clicks:	Control: 0.2797118847539016	Experiment: 0.3097463284379172
Variance clicks:	Control: 0.20147314628132165	Experiment: 0.2138035404571471
Levene's test statistic: 6.862145331711584
p-value: 0.008825098914964904
Variances are significantly different
T-test statistic: 2.615440020788211
p-value: 0.004466402814337101


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [73]:
control_click_rate = control.click.mean()
expected_experiment_clicks = control_click_rate * len(experiment)
print('Expected numer of clicks for the experiment group:', expected_experiment_clicks)

Expected numer of clicks for the experiment group: 838.0168067226891


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [85]:
#Your code here
n = len(experiment)
p = control_click_rate
var = n * p * (1-p)
std = np.sqrt(var)
print('Standard Deviation of a binomial distribution:', std)

experiment_clicks = experiment.click.sum()
z_score = (experiment_clicks - expected_experiment_clicks)/std
print('Number of standard deviations between actual and expected clicks', z_score)

Standard Deviation of a binomial distribution: 24.568547907005815
Number of standard deviations between actual and expected clicks 3.6625360854823588


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [87]:
import scipy.stats as stats
p_val = stats.norm.sf(z_score) 
print(p_val)
print((p_value/2)/p_val)


0.00012486528006951198
35.769773726136464


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: 
    We would reject the null hypothesis after either statistical test. 
    For the second test, the p-value is 35x smaller, meaning the chances of Type I error are much smaller and the effect size is bigger. 
    
    

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.